### Install dependencies and import all needed modules

In [ ]:
# Uninstall CoLab version of pytorch and install detectron2 compatiable version
!pip uninstall torch torchvision torchtext torchaudio -y
!pip install torch==1.10.1 torchvision==0.11.2 torchaudio==0.10.1

# Install pyyaml (version 5.1)
!pip install pyyaml==5.1

# Import torch
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

# Ensure detectron2 instalation matches pytorch version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

⚠️  **Attention:** After the installation of detectron2 in the above cell, **you may need to restart the runtime in Colab**. You can do this by selecting `Runtime > Restart Runtime` from the task bar, or by running the command `exit(0)`


After restarting the runtime, you will need to rerun the above cells. You should only have to do this once.


Once detectron has been successfully installed, you can continue to the next cells.


In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

## Model Selection
Select the model you would like to use to train

In [ ]:
# Use the following strings to select the model you want to work with:
# Resnet50: "keypoint_rcnn_R_50_FPN_3x"
# Resnet101: "keypoint_rcnn_R_101_FPN_3x"
# Resnext101: "keypoint_rcnn_X_101_32x8d_FPN_3x"

model_string = "keypoint_rcnn_X_101_32x8d_FPN_3x"

In [ ]:
REPO_LOC = "/content/drive/MyDrive/Trajectories/rhexis-trajectory"
DATASET_LOC = "/content/drive/MyDrive/Trajectories/datasets/Dataset_1000"
OUTPUT_LOC = "/content/drive/MyDrive/Trajectories/checkpoints/" + model_string

assert REPO_LOC is not None, "Must specify a repository location"
assert DATASET_LOC is not None, "Must specify a dataset location"
assert OUTPUT_LOC is not None, "Must specify a output location"

# Create the OUTPUT_LOC directory if it does not exist
import os
if not os.path.isdir(OUTPUT_LOC):
  os.makedirs(OUTPUT_LOC)

In [ ]:
# Import detectron2
import detectron2
from detectron2.utils.logger import setup_logger

# Set up detectron2 logger
setup_logger()

# Import common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# Import detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Import custom utility functions
import sys
sys.path.insert(0,f"{REPO_LOC}/Keypoint_Detection")
import utils

#Run frame-by-frame inference on test video using best model checkpoint:

In [ ]:
# Run frame-by-frame inference demo on video with demo.py" in the Detectron2 repo. 
# Note: in tutorial, panoptic segmentation of 5 second video took 3-4 minutes to run
!git clone https://github.com/facebookresearch/detectron2

# Initialize parameters for detectron2/demo/demo.py

# --config-file
# Save cfg as config.yaml in cfg.OUTPUT_DIR
with open(f"{cfg.OUTPUT_DIR}/config.yaml", 'w') as writer:
  writer.write(cfg.dump())
config_file = f"{cfg.OUTPUT_DIR}/config.yaml"
print("config_file: ", config_file, '\n')

# --video-input
# test video = DATASET_LOC/Cataract_Coach7_5s.mp4
test_video = DATASET_LOC + '/video.mp4'
print("test_video: ", test_video, '\n')

# --output
# Save to OUTPUT_LOC for now
video_output = OUTPUT_LOC + "/output.mp4"
print("video_output: ", video_output, '\n')

# --opts MODEL.WEIGHTS
# Assume for now that the best X101 model checkpoint trained with data augmentation on dataset 1000 is the overall best model checkpoint
# Will copy X101_dataset1000_no_augment_iter3499_best_model.pth to cfg.OUTPUT_DIR
best_model = 'X101_dataset1000_no_augment_iter3499_best_model.pth'
!cp $OUTPUT_LOC/$best_model $cfg.OUTPUT_DIR
print("Contents of cfg.OUTPUT_DIR: ")
!ls $cfg.OUTPUT_DIR

best_model_weights = cfg.OUTPUT_DIR + '/' + best_model
print("best_model_weights: ", best_model_weights, '\n')

In [ ]:
# Run inference
# Note: this is currently BROKEN for x264 codec but ok for mp4. See https://github.com/facebookresearch/detectron2/issues/2901 for workaround.

!python detectron2/demo/demo.py --config-file $config_file --video-input $test_video --confidence-threshold 0.95 --output $video_output --opts MODEL.WEIGHTS $best_model_weights